In [98]:
benchmarks = ['gcc', 'lbm', 'mcf', 'namd', 'xalancbmk']
complete_result = {}

In [104]:
def extract_counts(file_path):
    counters = {
        'mem_inst_retired.any': [], 
        'branch-instructions': [],
        'branch-misses': [], 
        'L1-icache-load-misses': [], 
        'L1-dcache-loads': [], 
        'L1-dcache-load-misses': [], 
        'L2-loads': [],
        'L2-load-misses': [], 
        'LLC-loads': [], 
        'LLC-load-misses': [], 
        'cycles': [], 
        'instructions': [],
    }
    with open(file_path, 'r') as f:
        for line in f:
            if line.startswith('#'):
                continue
            value, key = line.split()[1:3]
            counters[key].append(float(value.replace(',', '')))
            
    # Sanity Check
    lengths = [len(counters[key]) for key in counters]
    for i in range(1, len(lengths)):
        assert lengths[i] == lengths[i-1]
    print(f'Number of Count samples: {lengths[0]}')
    return counters

def extract_power(file_path):
    counters = {
        'joule': [], 
        'time': []
    }
    with open(file_path, 'r') as f:
        for line in f:
            if line.startswith('#'):
                continue
            time, joule = line.split()[0:2]
            counters['joule'].append(float(joule))
            counters['time'].append(float(time))
    
    # Sanity Check
    lengths = [len(counters[key]) for key in counters]
    for i in range(1, len(lengths)):
        assert lengths[i] == lengths[i-1]
    print(f'Number of Power samples: {lengths[0]}')
    return counters

def extract_results(counters):
    results = {
        'IPC': [],
        'branch-pred-acc': [],
        'L1-cahce-acc': [],
        'L2-cache-acc': [],
        'L3-cache-acc': [],
        'L1-hit-rate': [],
        'L2-hit-rate': [],
        'L3-hit-rate': [],
        'memory-reqs': [],
        'power': [],
    }
    normal_len = len(counters['instructions'])
    for i in range(normal_len):
        results['IPC'].append(counters['instructions'][i] / counters['cycles'][i])
        results['branch-pred-acc'].append(1 - counters['branch-misses'][i] / counters['branch-instructions'][i])
        results['L1-cahce-acc'].append(counters['L1-dcache-loads'][i])
        results['L2-cache-acc'].append(counters['L2-loads'][i])
        results['L3-cache-acc'].append(counters['LLC-loads'][i])
        results['L1-hit-rate'].append(1 - counters['L1-dcache-load-misses'][i] / counters['L1-dcache-loads'][i])
        results['L2-hit-rate'].append(1 - counters['L2-load-misses'][i] / counters['L2-loads'][i])
        results['L3-hit-rate'].append(1 - counters['LLC-load-misses'][i] / counters['LLC-loads'][i])
        L1_hits = counters['L1-dcache-loads'][i] - counters['L1-dcache-load-misses'][i]
        L2_hits = counters['L2-loads'][i] - counters['L2-load-misses'][i]
        L3_hits = counters['LLC-loads'][i] - counters['LLC-load-misses'][i]
        results['memory-reqs'].append(counters['mem_inst_retired.any'][i] - L1_hits - L2_hits - L3_hits)
    
    power_len = len(counters['joule'])
    for i in range(power_len-2):
        results['power'].append(counters['joule'][i] / (counters['time'][i+1] - counters['time'][i]))
        
    return results

In [108]:
for benchmark in benchmarks:
    print("--------------------------")
    print(f"Processing {benchmark}......")
    counters = extract_counts(f'./math/{benchmark}-math.count')
    power_counters = extract_power(f'./power/{benchmark}-phy.count')
    counters.update(power_counters)
    results = extract_results(counters)
    complete_result[benchmark] = results

--------------------------
Processing gcc......
Number of Count samples: 274
Number of Power samples: 280
--------------------------
Processing lbm......
Number of Count samples: 526
Number of Power samples: 528
--------------------------
Processing mcf......
Number of Count samples: 238
Number of Power samples: 240
--------------------------
Processing namd......
Number of Count samples: 476
Number of Power samples: 478
--------------------------
Processing xalancbmk......
Number of Count samples: 1377
Number of Power samples: 1351


In [97]:
results

{'IPC': [2.1705508794232675,
  1.6427032620435411,
  1.6959883059415937,
  1.6867457137278428,
  1.8258158604017527,
  1.8310544713529289,
  1.651610914660184,
  1.69242295372258,
  1.6542021158380682,
  1.6269770860125548,
  1.569009191521164,
  1.7141129749310327,
  1.6018511579982762,
  1.6951122856814385,
  1.3305121808855223,
  1.452680057596681,
  1.4458805586583885,
  1.4137756858750778,
  1.2868697318219464,
  1.3329045974305604,
  1.3673246086790625,
  1.2722421724736013,
  1.429923779814114,
  1.3897034343142445,
  1.9785795638322947,
  1.6189156913087952,
  1.4152752626552552,
  1.480500406040054,
  1.4994686146599658,
  1.4142919978563457,
  1.464247940070062,
  1.352173891878869,
  1.3977762901164739,
  1.6614213517844587,
  1.4605864598626976,
  1.6742587774610942,
  1.353381808680911,
  1.5883521015245288,
  1.4130925591119894,
  1.693208313200539,
  1.4440762627150274,
  1.5353491403548294,
  1.5399102384280523,
  1.6000153651014042,
  1.6007685115987484,
  1.5580471397